In [1]:
%pip install torch 
%pip install torchvision
%pip install transformers
%pip install meson

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://bart.sec.samsung.net/artifactory/api/pypi/pypi-remote/simple, https://pypi.python.org/simple
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://bart.sec.samsung.net/artifactory/api/pypi/pypi-remote/simple, https://pypi.python.org/simple
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://bart.sec.samsung.net/artifactory/api/pypi/pypi-remote/simple, https://pypi.python.org/simple
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://bart.sec.samsung.net/artifactory/api/pypi/pypi-remote/simple, https://pypi.python.org/simple
Note: 

In [2]:
from pathlib import Path
nntrainer_path = Path("./nntrainer")
ernie_path=Path("ERNIE")

if not nntrainer_path.is_dir():
    print("Start Download NNTrainer")
    !git clone https://github.com/DonghakPark/nntrainer.git
else:
    print("NNTrainer Already Exist")
    
if not ernie_path.is_dir():
    print("Start Download ERNIE")
    ernie_path.mkdir(parents=True, exist_ok=True)
    !hf download baidu/ERNIE-4.5-21B-A3B-Thinking --local-dir ./ERNIE
else:
    print("ERNIE Already Exist")

NNTrainer Already Exist
ERNIE Already Exist


In [3]:
import torch
import sys
import numpy as np
import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM

/home/user/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print("Python:", sys.version.split()[0])
print("Torch :", torch.__version__, "CUDA available:", torch.cuda.is_available())
print("Transformers:", transformers.__version__)

Python: 3.12.9
Torch : 2.9.1+cu128 CUDA available: False
Transformers: 4.57.1


In [5]:
data_dtype = "float32"
model_path = "./ERNIE"
output_name = "./erine_21b_fp32.bin"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, dtype="float32", trust_remote_code=True)
print("===Model Load Complete===")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
Loading checkpoint shards: 100%|██████████| 9/9 [00:15<00:00,  1.76s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


===Model Load Complete===


In [6]:
def save_ernie_moe_for_nntrainer(params, config, dtype, file):  
    """Convert and save weights as nntrainer format for multi-head attention model"""  

    n_layers = config.num_hidden_layers
    n_experts = config.num_experts
      
    def save_weight(weight_name, is_transpose=False):
        print(weight_name, params[weight_name].shape, "dtype = ", dtype )
        if is_transpose:
            np.array(params[weight_name].permute(1,0), dtype=dtype).tofile(file)  
        else:
            np.array(params[weight_name], dtype=dtype).tofile(file)  

    def save_projection(layer_name, proj_name):  
        save_weight(f"{layer_name}{proj_name}.weight", True)  

    def save_attention(layer_name):  
        """Save attention layer weights"""  
          
        # Save Q/K/V/O projections using helper  
        for proj in ["q_proj", "k_proj", "v_proj", "o_proj"]:  
            save_projection(layer_name, f"self_attn.{proj}")  
            proj_norm_name = f"{layer_name}self_attn.{proj[0]}_norm.weight"
            if proj_norm_name in params:
                save_weight(proj_norm_name)

    def save_feed_forward(layer_name):  
        """Save feed forward layer weights"""
        
        # First Dense Layer
        if layer_name == "model.layers.0.":
            for proj in ["up_proj", "gate_proj", "down_proj"]:  
                save_projection(layer_name, f"mlp.{proj}")  

        else:
            save_weight(f"{layer_name}mlp.gate.weight", True)
            save_weight(f"{layer_name}mlp.moe_statics.e_score_correction_bias", False)
                        
            for proj in ["up_proj", "gate_proj", "down_proj"]:  
                    save_projection(layer_name, f"mlp.shared_experts.{proj}")  
                
            # Save MoE projections using helper  
            for expert_id in range(n_experts):
                for proj in ["up_proj", "gate_proj", "down_proj"]:  
                    save_projection(layer_name, f"mlp.experts.{expert_id}.{proj}")  


    ##### START HERE FROM INITIAL LAYER ################################
    ####################################################################
    
    # 1. Save embedding layer  
    save_weight("model.embed_tokens.weight")

    # 2. Process all layers  
    for layer_idx in range(n_layers):  
        layer_prefix = f"model.layers.{layer_idx}."  
        save_weight(f"{layer_prefix}input_layernorm.weight")  
        save_attention(layer_prefix)  
        save_weight(f"{layer_prefix}post_attention_layernorm.weight")  
        save_feed_forward(layer_prefix)  

    # 3. Save Norm Weights
    save_weight("model.norm.weight")  
    
    print("mode save Done")
    
    ##### SAVE END HERE ################################################
    ####################################################################

########################################################################################
with open(output_name, "wb") as f_model :
        save_ernie_moe_for_nntrainer(model.state_dict(), config, data_dtype, f_model)

print("model save Success")


/tmp/ipykernel_1043/344963014.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  np.array(params[weight_name], dtype=dtype).tofile(file)


model.embed_tokens.weight torch.Size([103424, 2560]) dtype =  float32
model.layers.0.input_layernorm.weight torch.Size([2560]) dtype =  float32
model.layers.0.self_attn.q_proj.weight torch.Size([2560, 2560]) dtype =  float32


/tmp/ipykernel_1043/344963014.py:10: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  np.array(params[weight_name].permute(1,0), dtype=dtype).tofile(file)


model.layers.0.self_attn.k_proj.weight torch.Size([512, 2560]) dtype =  float32
model.layers.0.self_attn.v_proj.weight torch.Size([512, 2560]) dtype =  float32
model.layers.0.self_attn.o_proj.weight torch.Size([2560, 2560]) dtype =  float32
model.layers.0.post_attention_layernorm.weight torch.Size([2560]) dtype =  float32
model.layers.0.mlp.up_proj.weight torch.Size([12288, 2560]) dtype =  float32
model.layers.0.mlp.gate_proj.weight torch.Size([12288, 2560]) dtype =  float32
model.layers.0.mlp.down_proj.weight torch.Size([2560, 12288]) dtype =  float32
model.layers.1.input_layernorm.weight torch.Size([2560]) dtype =  float32
model.layers.1.self_attn.q_proj.weight torch.Size([2560, 2560]) dtype =  float32
model.layers.1.self_attn.k_proj.weight torch.Size([512, 2560]) dtype =  float32
model.layers.1.self_attn.v_proj.weight torch.Size([512, 2560]) dtype =  float32
model.layers.1.self_attn.o_proj.weight torch.Size([2560, 2560]) dtype =  float32
model.layers.1.post_attention_layernorm.weigh